## 5.5 MyFinance

Aplikacja MyFinance wykorzystuje `TabLayout Navigation` z `ViewPager2`. Dane dostarczymy jak w poprzednich przykładach przez obiekt `dataProvider` zawierający hardcodowane przykładowe pozycje. Aplikacja zawiera trzy ekrany pokazujące stan oszczędności oraz rachunki - ekran główny zawiera podsumowanie. Dodamy customową animację do `TabLayout.Tab` oraz `DonutChart` podsumowujący stan naszych finansów oraz kilka elementów urozmaicających wygląd naszej aplikacji. Sam layout aplikacji bazujev na przykładzie z **Android Open Project** [link](https://github.com/android/compose-samples/tree/main/Rally)

<table><tr><td><img src="https://media1.giphy.com/media/49RzyXBmRBa59Wi47n/giphy.gif?cid=790b76111b9b9991bd4197bf8d7295e517603d04f8ba2992&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media1.giphy.com/media/zbqykvBYncTRQLCAja/giphy.gif?cid=790b7611d47e3d6d46e7b736ac29fe3cc9bc281aed07c1c0&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media3.giphy.com/media/BbcDIUyV6IodX2yGVx/giphy.gif?cid=790b7611d1bf2abf7deacc8ad5a71a6a13adfbc9f4344954&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

Nasza aplikacja będzie posiadała zdefiniowany layout tylko w orientacji `portrait`, więc zablookujemy możliwość zmiany w plik `AndroidManifest.xml` - do aktywności dodamy

```xml
android:screenOrientation="portrait"
```

### **TabLayout i ViewPager2**

Aplikację rozpoczniemy od dodania nawigacji. Dodajmy trzy fragmenty (`BlankFragment`) - `OverviewFragment`, `AccountsFragment`, `BillsFragment`

In [ ]:
public class AccountsFragment extends Fragment {

    @Override
    public View onCreateView(LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_template, container, false);
    }
}

public class BillsFragment extends Fragment {

    @Override
    public View onCreateView(LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_template, container, false);
    }
}

public class OverviewFragment extends Fragment {

    @Override
    public View onCreateView(LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_template, container, false);
    }
}

W layoutach chwilowo zmieniam tylko text pola `TextView` na nazwę fragmentu. Ponieważ nasza aplikacja będzie posiadać tylko jeden motyw usuwamy plik `themes.xml(night)` do pliku `colors.xml` dodajemy kilka kolorów

```xml
<color name="green_500">#FF1EB980</color>
<color name="dark_blue_900">#FF26282F</color>
<color name="dark_blue_500">#FF004940</color>
```

Następnie modyfikujemy plik `themes.xml`

```xml
<style name="Theme.MyFinanceAppKotlin" parent="Theme.MaterialComponents.DayNight.NoActionBar">
    <!-- Primary brand color. -->
    <item name="colorPrimary">@color/dark_blue_500</item>
    <item name="colorPrimaryVariant">@color/dark_blue_900</item>
    <item name="colorOnPrimary">@color/teal_700</item>
    <!-- Secondary brand color. -->
    <item name="colorSecondary">@color/teal_700</item>
    <item name="colorSecondaryVariant">@color/teal_200</item>
    <item name="colorOnSecondary">@color/white</item>
    <!-- Status bar color. -->
    <item name="android:statusBarColor" tools:targetApi="l">?attr/colorPrimaryVariant</item>
    <!-- Customize your theme here. -->
</style>
```

Aplikacja nie będzie wykorzystywać `ActionBar`, więc jako `parent` podajemy `NoActionBar`.

Nasz `ViewPager2` dodamy bezpośrednio do `MainActivity` - zmodyfikujmy layout

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <com.google.android.material.tabs.TabLayout
        android:id="@+id/tabLayout"
        android:layout_width="match_parent"
        android:background="@color/dark_blue_900"
        app:tabTextColor="@color/white"
        android:layout_height="wrap_content"
        app:tabInlineLabel="true"
        app:tabIconTint="@color/teal_200"
        app:tabMode="fixed"
        app:tabGravity="start"
        app:tabMaxWidth="0dp" />

    <androidx.viewpager2.widget.ViewPager2
        android:id="@+id/viewPager"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"/>

</LinearLayout>
```

Kilka elementów zastosowanych w `TabLayout`
- `tabMode` - sposób w jaki mają zostać wyświetlone zakładki
    - `fixed` - tak jak zostaną podane, rozmiar zostanie automatycznie dostosowany
    - `scrollable` - zakładki nie będą dokowane na całą szerokość ekranu, tylko kilka zostanie wyświetlonych - dostęp do niewidocznych użytkownik może uzyskać przez przewijanie
- `tabGravity` - sposób rozmieszczenia zakładek
- `tabMaxWidth` - ustawienie maksymalnej szerokości - `0dp` dostosowuje rozmiar zakładki do jej zawartości
- `tabIconTint` - kolor wyświetlanej ikony
- `tabInlineLabel` - ustawienie ikony i tekstu w rzędzie - przyjmuje `true` lub `false`

`ViewPager2` zawiera `layout_height` ustawiony na `0dp` oraz `layout_weight = "1"` pozwala wypełnić pozostałą przestrzeń w `LinearLayout`
Dodajmy klasę `FinanceAdapter` do pliku `MainActivity`

In [ ]:
class FinanceAdapter extends FragmentStateAdapter {

    private final Fragment[] fragments = {
            new OverviewFragment(),
            new AccountsFragment(),
            new BillsFragment()
    };

    public FinanceAdapter(@NonNull FragmentActivity fragmentActivity) {
        super(fragmentActivity);
    }

    @NonNull
    @Override
    public Fragment createFragment(int position) {
        return fragments[position];
    }

    @Override
    public int getItemCount() {
        return fragments.length;
    }
}

Następnie utworzymy `ViewPager2` i `TabLayout` w metodzie `onCreate` głównej aktywności

In [ ]:
@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);

    ViewPager2 viewPager = findViewById(R.id.viewPager);
    viewPager.setAdapter(new FinanceAdapter(this));

    TabLayout tabLayout = findViewById(R.id.tabLayout);
    UiSetup.tabLayoutSetup(this, tabLayout, viewPager);
}

Następnym krokiem jest napisanie metody `tabLayoutSetup`, dodajmy do projektu nowy pakiet `util` i w nim utworzymy nowy plik `UiSetup`. Do pliku dodajmy tytuły naszych zakładek

In [ ]:
private static final int[] tabTitles = {R.string.overview, R.string.accounts, R.string.bills};

I do pliku `strings.xml`

```xml
<string name="overview_name">Overview</string>
<string name="accounts_name">Accounts</string>
<string name="bills_name">Bills</string>
```

Dodajmy metodę `tabLayoutSetup` do pliku `UiSetup`

In [ ]:
public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
    setupTabMediator(context, tabLayout, viewPager);
}

    private static void setupTabMediator(Context context, TabLayout tabLayout, ViewPager2 viewPager ){
        new TabLayoutMediator(tabLayout, viewPager,
                (tab, position) -> {
                        tab.setText(context.getString(tabTitles[position]));
                }
        ).attach();
    }

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/9IdIFNbyyewccfTTjG/giphy.gif?cid=790b76113f11b618eb6ce26cdf2e17faf07d98203cdf7e21&rid=giphy.gif&ct=g" width="150" />

Dodajmy acony do zakładek - do katalogu `drawable` dodajemy trzy **vector assets**

`ic_bills`
```xml
<vector android:height="24dp" android:tint="#FFFFFF"
    android:viewportHeight="24" android:viewportWidth="24"
    android:width="24dp" xmlns:android="http://schemas.android.com/apk/res/android">
    <path android:fillColor="@android:color/white" android:pathData="M12.5,6.9c1.78,0 2.44,0.85 2.5,2.1h2.21c-0.07,-1.72 -1.12,-3.3 -3.21,-3.81V3h-3v2.16c-0.53,0.12 -1.03,0.3 -1.48,0.54l1.47,1.47c0.41,-0.17 0.91,-0.27 1.51,-0.27zM5.33,4.06L4.06,5.33 7.5,8.77c0,2.08 1.56,3.21 3.91,3.91l3.51,3.51c-0.34,0.48 -1.05,0.91 -2.42,0.91 -2.06,0 -2.87,-0.92 -2.98,-2.1h-2.2c0.12,2.19 1.76,3.42 3.68,3.83V21h3v-2.15c0.96,-0.18 1.82,-0.55 2.45,-1.12l2.22,2.22 1.27,-1.27L5.33,4.06z"/>
</vector>
```

`ic_accounts`
```xml
<vector android:height="24dp" android:tint="#FFFFFF"
    android:viewportHeight="24" android:viewportWidth="24"
    android:width="24dp" xmlns:android="http://schemas.android.com/apk/res/android">
    <path android:fillColor="@android:color/white" android:pathData="M11.8,10.9c-2.27,-0.59 -3,-1.2 -3,-2.15 0,-1.09 1.01,-1.85 2.7,-1.85 1.78,0 2.44,0.85 2.5,2.1h2.21c-0.07,-1.72 -1.12,-3.3 -3.21,-3.81V3h-3v2.16c-1.94,0.42 -3.5,1.68 -3.5,3.61 0,2.31 1.91,3.46 4.7,4.13 2.5,0.6 3,1.48 3,2.41 0,0.69 -0.49,1.79 -2.7,1.79 -2.06,0 -2.87,-0.92 -2.98,-2.1h-2.2c0.12,2.19 1.76,3.42 3.68,3.83V21h3v-2.15c1.95,-0.37 3.5,-1.5 3.5,-3.55 0,-2.84 -2.43,-3.81 -4.7,-4.4z"/>
</vector>
```

`ic_overview`
```xml
<vector android:height="24dp" android:tint="#FFFFFF"
    android:viewportHeight="24" android:viewportWidth="24"
    android:width="24dp" xmlns:android="http://schemas.android.com/apk/res/android">
    <path android:fillColor="@android:color/white" android:pathData="M4,10h3v7h-3z"/>
    <path android:fillColor="@android:color/white" android:pathData="M10.5,10h3v7h-3z"/>
    <path android:fillColor="@android:color/white" android:pathData="M2,19h20v3h-20z"/>
    <path android:fillColor="@android:color/white" android:pathData="M17,10h3v7h-3z"/>
    <path android:fillColor="@android:color/white" android:pathData="M12,1l-10,5l0,2l20,0l0,-2z"/>
</vector>
```

Zmodyfikujmy metodę `tabLayoutSetup`

In [ ]:
private static final int[] tabTitles = {R.string.overview, R.string.accounts, R.string.bills};
private static final int[] tabIcons = {
    R.drawable.ic_overview,
    R.drawable.ic_accounts, 
    R.drawable.ic_bills};


public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
    setupTabMediator(context, tabLayout, viewPager);
}

private static void setupTabMediator(Context context, TabLayout tabLayout, ViewPager2 viewPager ){
    new TabLayoutMediator(tabLayout, viewPager,
            (tab, position) -> {
                tab.setIcon(ContextCompat.getDrawable(context, tabIcons[position]));
                tab.setText(context.getString(tabTitles[position]));
                Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
            }
    ).attach();
}

Metoda `getIcon.setTint` pozwala zmienić kolor grafiki wektorowej - ustawiamy na białą. Nasza aplikacja teraz wygląda następująco.

<img src="https://media0.giphy.com/media/gGFILN8lnvSsCIacNt/giphy.gif?cid=790b7611d2de0256b614247faddd193614ba76fa00201eec&rid=giphy.gif&ct=g" width="150" />

Chcemy pokazać tylko ikonę nie zaznaczonej zakładki oraz ikonę z tekstem wybranej zakładce. Dodajmy metodę `tabSelection` w której zaimplementujemy metody `addOnTabSelectedListener`. 

`TabSelectionListener` obsługuje tylko zdarzenia `onTabSelected`, `onTabUnselected` i `onTabReselected`. W pierwszych dwóch zmienimy tekst zakładki

In [ ]:
private static final int[] tabTitles = {R.string.overview, R.string.accounts, R.string.bills};
private static final int[] tabIcons = {
    R.drawable.ic_overview,
    R.drawable.ic_accounts, 
    R.drawable.ic_bills};


public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
    setupTabMediator(context, tabLayout, viewPager);
    setupTabSelection(context, tabLayout, vg);
}

private static void setupTabMediator(Context context, TabLayout tabLayout, ViewPager2 viewPager ){
    new TabLayoutMediator(tabLayout, viewPager,
            tab.setIcon(ContextCompat.getDrawable(context, tabIcons[position]));
            if (position == 0) {
                tab.setText(context.getString(tabTitles[position]));
                Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
            }
    ).attach();
}

private static void setupTabSelection(Context context, TabLayout tabLayout) {
    tabLayout.addOnTabSelectedListener(new TabLayout.OnTabSelectedListener() {
        @Override
        public void onTabSelected(TabLayout.Tab tab) {
            tab.setText(context.getString(tabTitles[tab.getPosition()]));
            Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
        }

        @Override
        public void onTabUnselected(TabLayout.Tab tab) {
            tab.setText("");
        }

        @Override
        public void onTabReselected(TabLayout.Tab tab) {

        }
    });
}

<img src="https://media0.giphy.com/media/uzc9KIxaD4KinhS1GQ/giphy.gif?cid=790b7611153c63bbec947cab1bc09d6c638f7323bdc36e8c&rid=giphy.gif&ct=g" width="150" />

Musimy jeszcze zainicjować w odpowiedni sposób zakładki. Zmodyfikujmy metodę `setupTabLayoutMediator`. Tekst i kolor ikony ustawimy tylko dla `position == 0`, co odpowiada ekranowi głównemu aplikacji.

In [ ]:
private static void setupTabMediator(Context context, TabLayout tabLayout, ViewPager2 viewPager ){
    new TabLayoutMediator(tabLayout, viewPager,
            (tab, position) -> {
                tab.setIcon(ContextCompat.getDrawable(context, tabIcons[position]));
                if (position == 0) {
                    tab.setText(context.getString(tabTitles[position]));
                    Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
                }
            }
    ).attach();
}

<img src="https://media0.giphy.com/media/8wosma6sipcea16wxt/giphy.gif?cid=790b76114eedb95754c5e7f3b59ff15167d82c6bc0910831&rid=giphy.gif&ct=g" width="150" />

Dodajmy customową animację do zakładek. Użyjemy prostego skalowania - niezaznaczona zakładka będzie wyświetlać tylko przeskalowaną (70%) ikonę. Dodajmy metodę `initialTabsSetup`

In [ ]:
private static void initialTabsSetup(TabLayout tabLayout, ViewGroup vg) {
    for (int i = 0; i < vg.getChildCount(); i++) {
        if (i == tabLayout.getSelectedTabPosition())
            continue;
        View tab = vg.getChildAt(i);
        tab.setScaleX(tabScaleLow);
        tab.setScaleY(tabScaleLow);
    }
}

Skalować możemy tylko na obiektach `View` - bezpośrednio nie możemy tego zrobić na `TabLayout.Tab`, ponieważ nie rozszerza on `View`. Więc jako parametr metody podamy `ViewGroup`, który będzie naszym zawierał nasze zakładki. Chcemy skalować wszystkie elementy poza aktualnie zaznaczonym, dlatego mamy warunek `if (...) continue`. Następnie wywołajmy tą metodę w `tabLayoutSetup`

In [ ]:
public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
    setupTabMediator(context, tabLayout, viewPager);
    ViewGroup vg = (ViewGroup) tabLayout.getChildAt(0);
    initialTabsSetup(tabLayout, vg);
    setupTabSelection(context, tabLayout, vg);
}

Z `TabLayout` wywołujemy metodę `getChildAt()` wyciągając wszyskie `Tabs` przepisane do tego `TabLayout` i rzutujemy na `ViewGroup` - co pozwala użyć na `Tabs` animacji. Dodajmy metodę `setupAnimation` wykonującą prostą animację powiększania.

In [ ]:
private static void setupAnimation(View view, float scale, long duration) {
    view.animate()
            .scaleX(scale)
            .scaleY(scale)
            .setInterpolator(new FastOutSlowInInterpolator())
            .setDuration(duration)
            .start();
}

- `scaleX`, `scaleY` - skalowanie do zadanej wartości z przedziału (0.0 - 1.0)
- `setInterpolator` - umożliwia wykorzystanie podstawowych animacji
- `FastOutSlowInInterpolator` - wykorzysuje [tablicowanie](https://pl.wikipedia.org/wiki/Tablicowanie) z [krzywymi Beziera](https://pl.wikipedia.org/wiki/Krzywa_B%C3%A9ziera) do wykonania utworzenia animacji - tutaj zastosujemy skalowanie
- `setDuration` - długość animacji w milisekundach

Teraz wykorzystajmy tą metodę do zmiany rozmiarów zakładek po zaznaczeniu.

In [ ]:
public final class UiSetup {

    private UiSetup(){}

    private static final float tabScaleLow = 0.7f;
    private static final float tabScaleHigh = 1f;

    private static final int[] tabTitles = {R.string.overview, R.string.accounts, R.string.bills};
    private static final int[] tabIcons = {
        R.drawable.ic_overview, 
        R.drawable.ic_accounts, 
        R.drawable.ic_bills};

    public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
        setupTabMediator(context, tabLayout, viewPager);
        ViewGroup vg = (ViewGroup) tabLayout.getChildAt(0);
        initialTabsSetup(tabLayout, vg);
        setupTabSelection(context, tabLayout, vg);
    }

    private static void setupTabSelection(Context context, TabLayout tabLayout, ViewGroup vg) {
        tabLayout.addOnTabSelectedListener(new TabLayout.OnTabSelectedListener() {
            @Override
            public void onTabSelected(TabLayout.Tab tab) {
                tab.setText(context.getString(tabTitles[tab.getPosition()]));
                Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);

                setupAnimation(vg.getChildAt(tab.getPosition()), tabScaleHigh,
                        (long) context.getResources()
                               .getInteger(android.R.integer.config_mediumAnimTime));
            }

            @Override
            public void onTabUnselected(TabLayout.Tab tab) {
                tab.setText("");
                setupAnimation(vg.getChildAt(tab.getPosition()), tabScaleLow, 1L);
            }

            @Override
            public void onTabReselected(TabLayout.Tab tab) {

            }
        });
    }

    private static void setupAnimation(View view, float scale, long duration) {
        view.animate()
                .scaleX(scale)
                .scaleY(scale)
                .setInterpolator(new FastOutSlowInInterpolator())
                .setDuration(duration)
                .start();
    }

    private static void initialTabsSetup(TabLayout tabLayout, ViewGroup vg) {
        for (int i = 0; i < vg.getChildCount(); i++) {
            if (i == tabLayout.getSelectedTabPosition())
                continue;
            View tab = vg.getChildAt(i);
            tab.setScaleX(tabScaleLow);
            tab.setScaleY(tabScaleLow);
        }
    }

    private static void setupTabMediator(Context context, TabLayout tabLayout, ViewPager2 viewPager ){
        new TabLayoutMediator(tabLayout, viewPager,
                (tab, position) -> {
                    tab.setIcon(ContextCompat.getDrawable(context, tabIcons[position]));
                    if (position == 0) {
                        tab.setText(context.getString(tabTitles[position]));
                        Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
                    }
                }
        ).attach();
    }
}


<img src="https://media2.giphy.com/media/4sMOjoEDLmVGtiC7FD/giphy.gif?cid=790b761140e3aa2701d05ed0f4d943a6f3bf656a58a60e90&rid=giphy.gif&ct=g" width="150" />

### **Dane**

Dodajmy dane do naszego projektu

In [ ]:
public class Account {
    private final String name;
    private final String number;
    private final double amount;
    private final int color;

    public Account(String name, String number, double amount, int color) {
        this.name = name;
        this.number = number;
        this.amount = amount;
        this.color = color;
    }

    public String getName() {
        return name;
    }

    public String getNumber() {
        return number;
    }

    public double getAmount() {
        return amount;
    }

    public int getColor() {
        return color;
    }
}


In [ ]:
import java.time.LocalDate;

public class Bill {
    private final String name;
    private final LocalDate date;
    private final double amount;
    private final int color;

    public Bill(String name, LocalDate date, double amount, int color) {
        this.name = name;
        this.date = date;
        this.amount = amount;
        this.color = color;
    }

    public String getName() {
        return name;
    }

    public LocalDate getDate() {
        return date;
    }

    public double getAmount() {
        return amount;
    }

    public int getColor() {
        return color;
    }
}


In [ ]:
public final class DataProvider {
    private DataProvider() {}

    public static Account[] accounts = {
            new Account("Home savings", "1111111111", 23456.34, Color.BLUE),
            new Account("Car savings", "2222222222", 126578.99, Color.LTGRAY),
            new Account("Vacation", "3457733323", 9875.12, Color.MAGENTA),
            new Account("Emergency", "9488344443", 10000.77, Color.RED),
            new Account("Healthcare", "3243554434", 12345.00, Color.YELLOW),
            new Account("Shopping", "2947560007", 3456.56, Color.BLACK)
    };

    public static Bill[] bills = {
            new Bill("Bank Credit", LocalDate.of(2022, 9,22), 2300.0, Color.BLACK),
            new Bill("Tuition", LocalDate.of(2023, 2,10), 1200.0, Color.BLUE),
            new Bill("Rent", LocalDate.of(2022, 8,3), 1023.87, Color.YELLOW),
            new Bill("Loan", LocalDate.of(2022, 12,22), 334.0, Color.GRAY),
            new Bill("Car Repair", LocalDate.of(2023, 1,9), 982.33, Color.WHITE),
            new Bill("Dress Loan", LocalDate.of(2023, 5,18), 243.0, Color.MAGENTA)
    };

    public static double totalAccountsAmount = Arrays
            .stream(accounts)
            .map(Account::getAmount)
            .collect(Collectors.toList())
            .stream()
            .mapToDouble(Double::doubleValue)
            .sum();

    public static double totalBillsAmount = Arrays
            .stream(bills)
            .map(Bill::getAmount)
            .collect(Collectors.toList())
            .stream()
            .mapToDouble(Double::doubleValue)
            .sum();
}

Klasa `Account` zawiera pola
- `name` - nazwa konta
- `number` - numer rachunku
- `amount` - zebrana kwota
- `color` - kolor powiązany z kontem

Klasa `Bills`:
- `name` - nazwa rachunku
- `endDate` - data spłaty
- `amount` - kwota do spłaty
- `color` - kolor powiązany z rachunkiem

W obiekcie `DataProvider` znnajdują się:
- `accounts` - lista wszystkich kont
- `bills` - lista wszystkich rachunków
- `totalAccountsAmount` - całkowita kwota na wszystkich rachunkach
- `totalBillsAmount` - całkowita kwota wszystkich rachunków

### **AccountsFragment**

Ponieważ `AccountsFragment` i `BillsFragment` będą używać tego samego layoutu, zdefiniujemy jeden plik `xml` o nazwie `fragment_detail_info`, rozpocznijmy od dodania `RecyclerView`

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.AccountsFragment">
    
        <androidx.recyclerview.widget.RecyclerView
            android:id="@+id/recyclerView"
            android:layout_width="match_parent"
            android:layout_height="0dp"
            android:layout_marginTop="12dp"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toTopOf="parent" />
        
</androidx.constraintlayout.widget.ConstraintLayout>
```

Utwórzmy layout elementu `RecyclerView`

`recyclerview_item_view.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="4dp"
    android:background="@color/dark_blue_500">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="12px"
        android:layout_height="0dp"
        android:background="@color/white"
        android:layout_marginStart="8dp"
        android:layout_marginBottom="8dp"
        android:layout_marginTop="8dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/RVNumberTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:fontFamily="serif-monospace"
        android:padding="4dp"
        android:text="******1111"
        android:textColor="@color/white"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toBottomOf="@+id/RVNameTextView" />

    <TextView
        android:id="@+id/RVNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="8dp"
        android:fontFamily="sans-serif-smallcaps"
        android:padding="4dp"
        android:text="Savings"
        android:textColor="@color/white"
        android:textSize="18sp"
        android:textStyle="bold"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/RVValueTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:padding="16dp"
        android:text="1111111 zł"
        android:textColor="@color/white"
        android:textSize="30sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Pierwszym elementem jest `View`, będzie to pasek pokazujący kolor powiązany w danym kontem.

Będziemy wyświetlać kwoty, więc dodajmy odpowiednie formatowanie, do pakietu `utils` dodajmy plik `FormatterUtil`. W nim dodajmy jedno pole zawierające `NumberFormat`

In [ ]:
public final class FormatterUtil {
    private FormatterUtil(){}

    public static NumberFormat formatter = new DecimalFormat("#,###.##");
}

Następnie zaimplementujmy `AccountAdapter`

In [ ]:
public class AccountAdapter extends RecyclerView.Adapter<AccountAdapter.ViewHolder> {

    @NonNull
    @Override
    public ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new ViewHolder(LayoutInflater.from(parent.getContext()).inflate(
                R.layout.recyclerview_item_view, parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull ViewHolder holder, int position) {
        Account item = DataProvider.accounts[position];
        holder.bind(item);
    }

    @Override
    public int getItemCount() {
        return DataProvider.accounts.length;
    }

    public static class ViewHolder extends RecyclerView.ViewHolder {

        private final TextView nameTextView;
        private final TextView accountNumberTextView;
        private final View colorBar;
        private final TextView amountTextView;

        public ViewHolder(@NonNull View itemView) {
            super(itemView);
            nameTextView = itemView.findViewById(R.id.RVaccountNameTextView);
            accountNumberTextView = itemView.findViewById(R.id.RVaccountsNumberTextView);
            colorBar = itemView.findViewById(R.id.RVcolorBarView);
            amountTextView = itemView.findViewById(R.id.RVaccountValueTextView);
        }

        public void bind(Account item){
            nameTextView.setText(item.getName());
            accountNumberTextView.setText(
                new StringBuilder(item.getNumber())
                .replace(0, 6, "******").toString()
            );
            amountTextView.setText(String.format(
                "%s zł", 
                FormatterUtil.formatter.format(item.getAmount()))
                                  );
            colorBar.setBackgroundColor(item.getColor());
        }
    }
}

Do `AccountsFragment` dodajmy `ViewPager2` w metodzie `onViewCreated`

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    RecyclerView recyclerView = view.findViewById(R.id.recyclerView);
    recyclerView.setAdapter(new AccountAdapter());
    recyclerView.setLayoutManager(new LinearLayoutManager(this.getContext()));
}

<img src="https://media2.giphy.com/media/grRVpWcSBxxdbt727W/giphy.gif?cid=790b76116c546b317dcc43461641e6ff4d09490235131cfa&rid=giphy.gif&ct=g" width="150" />

Kolejnym elementem będzie `DonutChart` (wykorzystamy [doughnut](https://github.com/futuredapp/donut)), do zależnościach dodajemy

```kotlin
implementation("app.futured.donut:donut:2.2.2")
```

do layoutu `fragment_detail_info` dodajemy `DonutProgressView` i dwa pola `TextView`

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.AccountsFragment">

        <TextView
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="@string/total"
            android:textColor="@color/white"
            android:textSize="26sp"
            app:layout_constraintBottom_toTopOf="@+id/totalAmountTextView"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent" />

        <TextView
            android:id="@+id/totalAmountTextView"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="100000 zł"
            android:textColor="@color/white"
            android:textSize="38sp"
            android:fontFamily="serif"
            app:layout_constraintBottom_toBottomOf="@+id/donut_viewAccount"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <app.futured.donut.DonutProgressView
            android:id="@+id/donut_viewAccount"
            android:layout_width="300dp"
            android:layout_height="300dp"
            app:donut_bgLineColor="@color/white"
            app:donut_gapAngle="20"
            app:donut_gapWidth="4"
            app:donut_animationDuration="1500"
            app:donut_strokeWidth="6dp"
            android:layout_marginTop="16dp"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <androidx.recyclerview.widget.RecyclerView
            android:id="@+id/recyclerView"
            android:layout_width="match_parent"
            android:layout_height="0dp"
            android:layout_marginTop="12dp"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/donut_viewAccount" />


</androidx.constraintlayout.widget.ConstraintLayout>
```



W `AccountsFragment` dodajemy obsługę `DonutProgressView` - musimy przekazać dane jako listę `DonutSection`. `DonutSection` przyjmuje trzy argumenty
- `name` - nazwę
- `color` - kolor
- `amount` - wartość z przedziału 0.0 - 1.0

In [ ]:
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

    RecyclerView recyclerView = view.findViewById(R.id.recyclerView);
    recyclerView.setAdapter(new AccountAdapter());
    recyclerView.setLayoutManager(new LinearLayoutManager(this.getContext()));

    DonutProgressView donut = view.findViewById(R.id.donut_viewAccount);
    List<DonutSection> values = new ArrayList<>();
    for (Account item : DataProvider.accounts){
        values.add(
                new DonutSection(
                        item.getName(),
                        item.getColor(),
                        ((float) item.getAmount() / (float) DataProvider.totalAccountsAmount))
        );
    }
    donut.submitData(values);

    TextView totalAmountTextView = view.findViewById(R.id.totalAmountTextView);
    totalAmountTextView.setText(String.format(
        "%s zł", 
        FormatterUtil.formatter.format(DataProvider.totalAccountsAmount))
    );
}

W efekcie otrzymamy

<img src="https://media3.giphy.com/media/V6olKtypWuObTJXxmz/giphy.gif?cid=790b761116be30abedbc5c9f9c7a309c74243ac9e1060564&rid=giphy.gif&ct=g" width="150" />

### **BillsFragment**

Do `FormatterUtil` dodajmy `DateTimeFormatter` pozwalający sformatować datę

In [ ]:
public final class FormatterUtil {
    private FormatterUtil(){}

    public static NumberFormat formatter = new DecimalFormat("#,###.##");
    public static DateTimeFormatter dateFormatter = DateTimeFormatter.ofPattern("yyyy MMM dd");
}

Przejdźmy do `BillAdapter`

In [ ]:
public class BillAdapter extends RecyclerView.Adapter<BillAdapter.ViewHolder> {

    @NonNull
    @Override
    public ViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new ViewHolder(LayoutInflater.from(parent.getContext()).inflate(
                R.layout.recyclerview_item_view, parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull ViewHolder holder, int position) {
        Bill item = DataProvider.bills[position];
        holder.bind(item);
    }

    @Override
    public int getItemCount() {
        return DataProvider.accounts.length;
    }

    public static class ViewHolder extends RecyclerView.ViewHolder {

        private final TextView nameTextView;
        private final TextView endDateTextView;
        private final View colorBar;
        private final TextView amountTextView;

        public ViewHolder(@NonNull View itemView) {
            super(itemView);
            nameTextView = itemView.findViewById(R.id.RVaccountNameTextView);
            endDateTextView = itemView.findViewById(R.id.RVaccountsNumberTextView);
            colorBar = itemView.findViewById(R.id.RVcolorBarView);
            amountTextView = itemView.findViewById(R.id.RVaccountValueTextView);
        }

        public void bind(Bill item){
            nameTextView.setText(item.getName());
            endDateTextView.setText(item.getDate().format(FormatterUtil.dateFormatter));
            amountTextView.setText(String.format(
                "- %s zł", 
                FormatterUtil.formatter.format(item.getAmount()))
        );
            colorBar.setBackgroundColor(item.getColor());
        }
    }
}


Korzystamy w tego samego layoutu co przy implementacji `AccountsFragment`

In [ ]:
public class BillsFragment extends Fragment {

    @Override
    public View onCreateView(LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_detail_info, container, false);
    }

    @Override
    public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
        super.onViewCreated(view, savedInstanceState);
        RecyclerView recyclerView = view.findViewById(R.id.recyclerView);
        recyclerView.setAdapter(new BillAdapter());
        recyclerView.setLayoutManager(new LinearLayoutManager(this.getContext()));

        DonutProgressView donut = view.findViewById(R.id.donut_viewAccount);
        List<DonutSection> values = new ArrayList<>();
        for (Bill item : DataProvider.bills){
            values.add(
                    new DonutSection(
                            item.getName(),
                            item.getColor(),
                            ((float) item.getAmount() / (float) DataProvider.totalBillsAmount))
            );
        }
        donut.submitData(values);

        TextView totalAmountTextView = view.findViewById(R.id.totalAmountTextView);
        totalAmountTextView.setText(String.format(
            "- %s zł", 
            FormatterUtil.formatter.format(DataProvider.totalBillsAmount))
        );
    }
}

### **OverviewFragment**

Layout `OverviewFragment` będzie się opierał na `CardView`, więc musimy dodać odpowiednią zależność do aplikacji

```kotlin
implementation "androidx.cardview:cardview:1.0.0"
```

Sam layout nie zawiera żadnych nowych elementów

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    android:orientation="vertical"
    tools:context=".fragments.OverviewFragment">

    <androidx.cardview.widget.CardView
        android:id="@+id/cardView0"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="16dp"
        android:layout_marginStart="16dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical">

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:text="@string/alerts"
                android:textColor="@color/white"
                android:textSize="12sp" />

            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:orientation="horizontal">

                <TextView
                    android:layout_width="0dp"
                    android:layout_weight="1"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="16dp"
                    android:text="@string/no_alerts"
                    android:textColor="@color/white"
                    android:textSize="24sp" />

                <Button
                    android:id="@+id/seeMoreButton"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:backgroundTint="@android:color/holo_red_light"
                    android:textColor="@color/design_default_color_on_secondary"
                    android:layout_marginEnd="8dp"
                    android:text="@string/see_more" />
            </LinearLayout>
        </LinearLayout>
    </androidx.cardview.widget.CardView>

    <androidx.cardview.widget.CardView
        android:id="@+id/cardView1"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginTop="8dp"
        android:layout_marginStart="16dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical"
            tools:context=".fragments.OverviewFragment">


            <TextView
                android:id="@+id/accountNameTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="8dp"
                android:text="@string/accounts"
                android:textColor="@color/white"
                android:textSize="12sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:id="@+id/accountTotalAmountTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginTop="4dp"
                android:gravity="center"
                android:text="11111111 zł"
                android:textColor="@color/white"
                android:textSize="36sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountNameTextView" />

            <androidx.recyclerview.widget.RecyclerView
                android:id="@+id/accountsRecyclerView"
                android:layout_width="match_parent"
                android:layout_height="0dp"
                android:layout_weight="1"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountTotalAmountTextView" />

            <Button
                android:id="@+id/accountsButton"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                android:text="@string/go_to_accounts"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountsRecyclerView"
                android:backgroundTint="@android:color/holo_red_light"
                android:textColor="@color/design_default_color_on_secondary"/>
        </LinearLayout>


    </androidx.cardview.widget.CardView>

    <androidx.cardview.widget.CardView
        android:id="@+id/cardView2"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginStart="16dp"
        android:layout_marginTop="8dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical"
            tools:context=".fragments.OverviewFragment">


            <TextView
                android:id="@+id/billsNameTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="8dp"
                android:text="@string/bills"
                android:textColor="@color/white"
                android:textSize="12sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:id="@+id/billsTotalAmountTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="4dp"
                android:gravity="center"
                android:text="- 21999.99 zł"
                android:textColor="@color/white"
                android:textSize="36sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="@id/billsNameTextView" />

            <androidx.recyclerview.widget.RecyclerView
                android:id="@+id/billsRecyclerView"
                android:layout_width="match_parent"
                android:layout_height="0dp"
                android:layout_weight="1"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/billsTotalAmountTextView" />

            <Button
                android:id="@+id/billsButton"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                android:text="@string/go_to_bills"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/billsRecyclerView"
                android:backgroundTint="@android:color/holo_red_light"
                android:textColor="@color/design_default_color_on_secondary"/>
        </LinearLayout>
    </androidx.cardview.widget.CardView>


</LinearLayout>
```


Do metody `onCreateView` dodajmy obsługę wszystkich elementów. Zacznijmy od `RecyclerVew`

In [ ]:
RecyclerView accountsRecyclerView = view.findViewById(R.id.accountsRecyclerView);
accountsRecyclerView.setLayoutManager(new LinearLayoutManager(getContext()));
accountsRecyclerView.setAdapter(new AccountAdapter());

RecyclerView billsRecyclerView = view.findViewById(R.id.billsRecyclerView);
billsRecyclerView.setLayoutManager(new LinearLayoutManager(getContext()));
billsRecyclerView.setAdapter(new BillAdapter());

Następnie mamy dwa pola `TextView` wyświetlające wartości wszystkich kont oraz rachunków

In [ ]:
TextView accountTotalAmountTextView = view.findViewById(R.id.accountTotalAmountTextView);
accountTotalAmountTextView.setText(String.format(
    "%s zł", 
    FormatterUtil.formatter.format(DataProvider.totalAccountsAmount))
);

TextView billsTotalAmountTextView = view.findViewById(R.id.billsTotalAmountTextView);
billsTotalAmountTextView.setText(String.format(
    "%s zł", 
    FormatterUtil.formatter.format(DataProvider.totalBillsAmount))
);

Kolejno dodajmy obsługę dwóch przycisków - w tym przykładzie dodamy przejście na konkretną pozycję w `ViewPager2` - znajduje się on w głównej aktywności, więc z poziomu fragmentu nie mamy do niego dostępu. Możemy obejść ten problem wykorzystując metod `requireActivity` oraz `findViewById`.

In [ ]:
Button goToAccountsButton = view.findViewById(R.id.accountsButton);
goToAccountsButton.setOnClickListener(v -> {
    ViewPager2 viewPager2 = requireActivity().findViewById(R.id.viewPager);
    viewPager2.setCurrentItem(1);
});

Button goToBillsButton = view.findViewById(R.id.billsButton);
goToBillsButton.setOnClickListener(v -> {
    ViewPager2 viewPager2 = requireActivity().findViewById(R.id.viewPager);
    viewPager2.setCurrentItem(2);
});

Ostatnim elementem będzie obsługa przycisku `SEE MORE` - tutaj wyświetlimy customowy `AlertDialog`. W pierwszsym kroku zdefiniujemy styl w pliku `themes.xml`

```xml
<style name="MyDialogTheme" parent="Theme.AppCompat.Light.Dialog.Alert">
    <item name="android:background">@color/dark_blue_900</item>
    <item name="android:textColor">@color/white</item>
</style>
```

Następnie utwórzmy layout

`alert_dialog.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout
    xmlns:android="http://schemas.android.com/apk/res/android"
    android:orientation="vertical"
    android:paddingLeft="20dp"
    android:paddingRight="20dp"
    android:layout_width="match_parent"
    android:background="@color/dark_blue_900"
    android:layout_height="match_parent">

    <TextView
        android:id="@+id/alertTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        android:textColor="@color/white"
        android:gravity="center"
        android:text="@string/no_alerts"/>
</LinearLayout>
```

Na koniec przechodzimy do implementacji metody `onClick`

In [ ]:
Button seeMoreButton = view.findViewById(R.id.seeMoreButton);
seeMoreButton.setOnClickListener(v -> {
    new AlertDialog.Builder(getContext(), R.style.MyDialogTheme)
            .setTitle(getString(R.string.alerts)) // tytuł
            .setView(getLayoutInflater().inflate(R.layout.alert_dialog, null)) // layout
            .setPositiveButton("OK", (dialogInterface, i) -> {}) // domyślny przycisk pozwalający na wyjście
            .create() // utwórz AlertDialog
            .show(); // wyświetl Dialog
});

Przekazanie `null` jako parametru metody `setView` generuje ostrzeżenie - w tym przykładzie pozbędziemy się go przez umieszczenie adnotacji `@SuppressLint("InflateParams")`.

In [ ]:
@SuppressLint("InflateParams")
@Override
public void onViewCreated(@NonNull View view, @Nullable Bundle savedInstanceState) {
    super.onViewCreated(view, savedInstanceState);

Możemy przetestować aplikację.

<table><tr><td><img src="https://media1.giphy.com/media/49RzyXBmRBa59Wi47n/giphy.gif?cid=790b76111b9b9991bd4197bf8d7295e517603d04f8ba2992&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media1.giphy.com/media/zbqykvBYncTRQLCAja/giphy.gif?cid=790b7611d47e3d6d46e7b736ac29fe3cc9bc281aed07c1c0&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media3.giphy.com/media/BbcDIUyV6IodX2yGVx/giphy.gif?cid=790b7611d1bf2abf7deacc8ad5a71a6a13adfbc9f4344954&rid=giphy.gif&ct=g" width="150" /></td></tr></table>